# 库文件

In [39]:
import pandas as pd
import numpy as np
import math

# 设定路径

In [40]:
train_filepath = 'train.csv'
test_filepath = "test.csv"


# 读取测试文件和测试文件

In [41]:
class clearinfo:
    def __init__(self,train_filepath,test_filepath):
        self.train_filepath = train_filepath
        self.test_filepath = test_filepath
    def getinfo_train(self):
        train_data = pd.read_csv(self.train_filepath,header=0)
        return train_data
    def getinfo_test(self):
        test_data = pd.read_csv(self.test_filepath,header=0)
        return test_data
info = clearinfo(train_filepath,test_filepath)

In [42]:
df_train = info.getinfo_train()
#df_train
#for row in df_train.iterrows():
#    print(row)

# 得到user_business矩阵函数
df 转换为 dic

In [43]:
def getUserBus(train_info):
    UB_dict = {}
    for row in train_info.itertuples():
        if row[1] not in UB_dict:
            UB_dict.setdefault(row[1],{})
        UB_dict[row[1]][row[2]]=row[4]
    return UB_dict
UB = getUserBus(df_train)

In [44]:
df_train = info.getinfo_train()
df_test  = info.getinfo_test()
#df_train = df_train.pivot_table(index = 'user_id',columns='business_id',values="stars")

# 构建共同评分向量

In [45]:
def getsameVec(user1,user2,uudic):
    user1_info = uudic[user1]
    user2_info = uudic[user2]
    #获取两位用户的评分向量
    sameBusiness = []
    for key in user1_info.keys():
        if key in user2_info.keys():
            sameBusiness.append(key)
    x = {}
    y = {}
    for ele in sameBusiness:
        x[ele]=user1_info[ele]
        y[ele]=user2_info[ele]
    return x,y
#getsameVec("kjeX2RXvW7RhBbD2QLd5jA","1O638BDK_fWuxgTVJwff-A",getUserBus(df_train))

In [46]:
#欧几里得距离
def euclidean(user1,user2,uudic):
    x,y=getsameVec(user1,user2,uudic)
    distance = 0
    for key in x.keys():
        distance+=(x[key]-y[key])**2
    try:
        distance = 1+math.sqrt(distance)
    except ZeroDivisionError:
        distance = 0
    return distance

#余弦相似度
def cosine(user1,user2,uudic):
    x, y= getsameVec(user1,user2,uudic)
    up=0
    down1=0
    down2=0
    for key in x.keys():
        up += x[key]*y[key]
        down1 += x[key]**2
        down2 +=y[key]**2
    down = (down1*down2)**0.5
    try:
        dis = up/down
    except ZeroDivisionError:
        dis=0
    return dis

#皮尔逊相关系数
def pearson(user1,user2,uudic):
    x,y =getsameVec(user1,user2,uudic)
    length = len(x)
    if length == 0:
        return 0
    sumx=0
    sumy=0
    for key in x.keys():
        sumx+=x[key]
        sumy+=y[key]
    mean1 = sumx/length
    mean2 = sumy/length
    up=0
    down1=0
    down2=0
    for key in x.keys():
        up += (x[key]-mean1)*(y[key]-mean2)
        down1 +=(x[key]-mean1)**2
        down2 +=(y[key]-mean2)**2
    down = (down1*down2)**0.5
    try:
        value = up / down
    except ZeroDivisionError:
        value = 0
    return value

# 得到用户两两相似度矩阵

In [47]:
euclidean("kjeX2RXvW7RhBbD2QLd5jA","1O638BDK_fWuxgTVJwff-A",getUserBus(df_train))

1.0

In [48]:
def getUU_cos(UB):
    UU_dict = {}
    for key in UB.keys():
        UU_dict.setdefault(key,{})
        for key_other in UB.keys():
            if not key == key_other:
                if cosine(key,key_other,UB) > 0:  
#使用余弦相似度，可替换函数为皮尔逊和欧式距离
                    UU_dict[key][key_other] = cosine(key,key_other,UB)
        #UU_dict[key] = sorted(UU_dict[key].items(),key = lambda x:x[1],reverse=True)
    return UU_dict  

def getUU_euc(UB):
    UU_dict = {}
    for key in UB.keys():
        UU_dict.setdefault(key,{})
        for key_other in UB.keys():
            if not key == key_other:
                if euclidean(key,key_other,UB) > 0:  
#使用欧式距离
                    UU_dict[key][key_other] = euclidean(key,key_other,UB)
        #UU_dict[key] = sorted(UU_dict[key].items(),key = lambda x:x[1],reverse=True)
    return UU_dict  

def getUU_pea(UB):
    UU_dict = {}
    for key in UB.keys():
        UU_dict.setdefault(key,{})
        for key_other in UB.keys():
            if not key == key_other:
                if pearson(key,key_other,UB) > 0:  
#使用皮尔逊
                    UU_dict[key][key_other] = pearson(key,key_other,UB)
        #UU_dict[key] = sorted(UU_dict[key].items(),key = lambda x:x[1],reverse=True)
    return UU_dict  

In [49]:
UU_cos = getUU_cos(UB) #相似用户矩阵_余弦相似度
UU_euc = getUU_euc(UB) #相似用户矩阵_欧氏距离
UU_pea = getUU_pea(UB) #相似用户矩阵_皮尔逊相似度
#UU

In [50]:
def preScore(user,business,UB,UU):
    up = 0
    down = 0
    for simiUser in UU[user].keys():
        if business in UB[simiUser].keys():
            up += UB[simiUser][business]*UU[user][simiUser]
            down += UU[user][simiUser]
    try:
        value = up/down
    except ZeroDivisionError:
        value = 0
    return  round(value,1)
preScore("48vRThjhuhiSQINQ2KV8Sw","LNGBEEelQx4zbfWnlc66cw",UB,UU_pea)

0

# 欧氏距离

In [53]:
def pre_result(test):
    lis = []
    for row in test.itertuples():
        lis.append(preScore(row[1],row[2],UB,UU_euc))
    df_temp = pd.DataFrame(lis,columns =["pre_stars"] )
    df = pd.concat([test,df_temp],axis=1)
    return df
pre_result(df_test).to_csv("Result1.csv")

# 余弦相似度

In [55]:
def pre_result(test):
    lis = []
    for row in test.itertuples():
        lis.append(preScore(row[1],row[2],UB,UU_cos))
    df_temp = pd.DataFrame(lis,columns =["pre_stars"] )
    df = pd.concat([test,df_temp],axis=1)
    return df
pre_result(df_test).to_csv("Result2.csv")

# 皮尔逊相似度

In [56]:
def pre_result(test):
    lis = []
    for row in test.itertuples():
        lis.append(preScore(row[1],row[2],UB,UU_pea))
    df_temp = pd.DataFrame(lis,columns =["pre_stars"] )
    df = pd.concat([test,df_temp],axis=1)
    return df
pre_result(df_test).to_csv("Result3.csv")
